In [1]:
import pyspark

In [2]:
sc = pyspark.SparkContext('local[*]')
txt = sc.textFile('file:////usr/share/doc/python/copyright')
big_list=range(10000)
rdd = sc.parallelize(big_list, 2)

In [ ]:
odds = rdd.filter(lambda x: x % 2 != 0)

In [ ]:
odds.take(5)

In [ ]:
odds.collect()
o

In [ ]:
odds.take(5)

In [ ]:
lines = sc.textFile("C://Users//nolfonzo//Spark//README.md")

In [ ]:
lines

In [ ]:
words = lines.flatMap(lambda s: s.split(" "))

In [ ]:
word_tuples = words.map(lambda s: (s, 1))

In [ ]:
word_tuples.collect()

In [ ]:
take(word_tuples)

In [ ]:
word_tuples.take(1)

In [ ]:
from pyspark.sql.functions import split, explode

In [ ]:
linesDF = spark.read.text("C://Users//nolfonzo//Spark//README.md")

In [ ]:
linesDF

In [ ]:
print(linesDF)

In [ ]:
linesDF.show()

In [ ]:
wordListDf = linesDF.select(split("value", " ").alias("words"))

In [ ]:
wordListDf.show()

In [ ]:
wordsDf = wordListDf.select(explode("words").alias("word"))

In [ ]:
wordsDf.show()

In [ ]:
wordCountDf = wordsDf.groupBy("word").count()

In [ ]:
wordCountDf.show()

In [ ]:
spark = SparkSession(sc)

In [ ]:
from pyspark.sql.session import SparkSession

In [ ]:
CREATE TABLE word_counts (word STRING)
USING csv
OPTIONS("delimiter"=" ")
LOCATION "/databricks-datasets/README.md"

In [ ]:
spark.sql("CREATE TABLE word_counts (word STRING) USING csv OPTIONS('delimiter'=' ') LOCATION '/databricks-datasets/README.md'")

In [ ]:
spark.sql("SELECT word, COUNT(word) AS count")

In [ ]:
spark.sql('CREATE TABLE word_counts (word STRING) USING csv OPTIONS("delimiter"=" ") LOCATION "/databricks-datasets/README.md"')

In [ ]:
myDF = spark.sql("SELECT word, COUNT(word) AS count FROM word_counts GROUP BY word")

In [ ]:
myDF.show()

In [ ]:
spark.sql("drop table word_counts")

In [ ]:
spark.sql('CREATE TABLE word_counts (word STRING) USING csv OPTIONS("delimiter"=" ") LOCATION "C://Users//nolfonzo//Spark//README.md"')


In [ ]:
spark.sql("SELECT word, COUNT(word) AS count FROM word_counts GROUP BY word").show()

In [ ]:
spark.sql('CREATE TABLE word_counts2 (word STRING) USING csv OPTIONS("delimiter"=" ") LOCATION "C://Users//nolfonzo//Spark//README.md"').show()

In [ ]:
spark.sql("select * from word_counts2").show()